<a href="https://colab.research.google.com/github/bmnds/uea-data-science-04-data-processing/blob/main/trabalho-final/Notebook_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pwd

/content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets


In [37]:
# Importando libs necessárias
import pandas as pd

# Extração

In [38]:
# Montar o diretório compartilhado do Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive


In [39]:
# Listar arquivos do diretório compartilhado
%cd MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets
!ls

/content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets
bweb_1t_AM_181120201549.zip
bweb_2t_AM_301120201245.zip
dados-bairros-manaus.zip
detalhe_votacao_secao_2020.zip
eleitorado_local_votacao_2020.zip
perfil_comparecimento_abstencao_2020.zip
perfil_eleitorado_2020.zip
perfil_eleitor_secao_2020_AM.zip
prestacao_contas_anual_partidaria_2020.zip
prestacao_de_contas_eleitorais_candidatos_2020.zip
votacao_candidato_munzona_2020.zip
votacao_secao_2020_AM.zip


# Transformação

In [40]:
!mkdir /content/work #diretório temporário de trabalho
!mkdir /content/datasets #diretório com os arquivos processados

mkdir: cannot create directory ‘/content/work’: File exists
mkdir: cannot create directory ‘/content/datasets’: File exists


## Dados de Boletins de Urna

In [41]:
# boletins de urna do 1o turno
!unzip -u bweb_1t_AM_181120201549.zip -d /content/work

Archive:  bweb_1t_AM_181120201549.zip
  inflating: /content/work/bweb_1t_AM_181120201549.csv  
  inflating: /content/work/leiame-boletimurnaweb.pdf  


In [42]:
!cp /content/work/leiame-boletimurnaweb.pdf /content/datasets/leiame-boletim_urna.pdf

In [43]:
df_bu_1t = pd.read_csv('/content/work/bweb_1t_AM_181120201549.csv', encoding="ISO-8859-1", delimiter=';')
df_bu_1t.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DT_BU_RECEBIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,CD_TIPO_URNA,DS_TIPO_URNA,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,DT_ABERTURA,DT_ENCERRAMENTO,QT_ELEITORES_BIOMETRIA_NH,DT_EMISSAO_BU,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
435108,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,40,464,1503,13,Vereador,43,PV,Partido Verde,15/11/2020 18:34:41,402,340,62,1,APURADA,1,Nominal,43235,THIAGO ALFAIA,1,1787953,485.676.700.230.804.355.,271.103,A59D0CF8,12/11/2020 14:57:00,13 - 464,#NULO#,15/11/2020 07:07:09,15/11/2020 17:01:09,0,15/11/2020 17:03:51,-1,-1
544092,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,58,980,1805,13,Vereador,33,PMN,Partido da Mobilização Nacional,15/11/2020 18:30:37,395,301,94,1,APURADA,1,Nominal,33011,DR VITOR VERÇOSA,1,1781459,021.025.392.166.212.553.,835.785,26212384,11/11/2020 08:25:00,13 - 980,#NULO#,15/11/2020 07:06:58,15/11/2020 17:08:22,0,15/11/2020 17:08:40,-1,-1
496439,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,98353,IRANDUBA,56,76,1376,13,Vereador,11,PP,PROGRESSISTAS,15/11/2020 20:37:59,196,170,26,1,APURADA,1,Nominal,11999,CHELLEY DA PESCA,2,1290765,249.825.390.634.788.915.,813.905,05FF9D8F,01/11/2020 15:21:00,13 - 76,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:57,0,15/11/2020 17:07:00,-1,-1
773799,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,68,562,1732,13,Vereador,51,PATRIOTA,Patriota,15/11/2020 18:31:53,413,343,70,1,APURADA,1,Nominal,51221,DR TADEU,1,1789233,536.183.625.489.907.557.,650.973,BBB0769F,05/11/2020 14:38:00,13 - 562,#NULO#,15/11/2020 07:00:00,15/11/2020 17:11:59,0,15/11/2020 17:13:59,-1,-1
293849,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,31,737,2127,13,Vereador,10,REPUBLICANOS,REPUBLICANOS,15/11/2020 18:58:31,459,365,94,1,APURADA,1,Nominal,10190,CAPITÃO CARPÊ ANDRADE,2,1788097,536.185.143.312.332.035.,881.952,D8A2C33A,05/11/2020 09:46:00,13 - 737,80,15/11/2020 07:00:00,15/11/2020 17:13:30,0,15/11/2020 17:14:54,-1,-1


In [44]:
df_bu_1t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848941 entries, 0 to 848940
Data columns (total 45 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   DT_GERACAO                   848941 non-null  object 
 1   HH_GERACAO                   848941 non-null  object 
 2   ANO_ELEICAO                  848941 non-null  int64  
 3   CD_TIPO_ELEICAO              848941 non-null  int64  
 4   NM_TIPO_ELEICAO              848941 non-null  object 
 5   CD_PLEITO                    848941 non-null  int64  
 6   DT_PLEITO                    848941 non-null  object 
 7   NR_TURNO                     848941 non-null  int64  
 8   CD_ELEICAO                   848941 non-null  int64  
 9   DS_ELEICAO                   848941 non-null  object 
 10  SG_UF                        848941 non-null  object 
 11  CD_MUNICIPIO                 848941 non-null  int64  
 12  NM_MUNICIPIO                 848941 non-null  object 
 13 

In [45]:
df_bu_1t.DS_CARGO_PERGUNTA.unique()

array(['Prefeito', 'Vereador'], dtype=object)

In [46]:
df_bu_1t.SG_PARTIDO.value_counts()

PSC              62642
AVANTE           60226
REPUBLICANOS     53694
PL               40921
PATRIOTA         40139
PP               36335
PT               34365
PSDB             33885
PODE             33417
PSL              33355
PTB              32379
MDB              31878
PMN              30224
PSB              29228
PSD              27816
DC               26548
CIDADANIA        26338
PROS             26234
PV               25449
#NULO#           24702
DEM              24324
PRTB             21962
PTC              21249
SOLIDARIEDADE    17075
PC do B          16626
PDT              12287
PMB               9386
NOVO              7568
PSOL              4880
REDE              2877
PSTU               932
Name: SG_PARTIDO, dtype: int64

In [47]:
df_bu_1t.NM_MUNICIPIO.value_counts()

MANAUS         590874
MANACAPURU      20977
ITACOATIARA     20787
PARINTINS       18377
TEFÉ            12425
                ...  
AMATURÁ          1116
ANAMÃ            1092
CANUTAMA         1033
ITAMARATI         800
JAPURÁ            583
Name: NM_MUNICIPIO, Length: 62, dtype: int64

In [48]:
df_bu_1t.DS_TIPO_VOTAVEL.unique()

array(['Nominal', 'Nulo', 'Branco', 'Legenda'], dtype=object)

In [49]:
df_bu_1t.DS_TIPO_URNA.unique()

array(['APURADA'], dtype=object)

In [50]:
df_bu_1t.DS_AGREGADAS.value_counts()

#NULO#     805387
80            585
73            551
50            467
61            423
            ...  
23 / 45        24
65             19
17 / 26        19
32             17
111            14
Name: DS_AGREGADAS, Length: 294, dtype: int64

In [51]:
df_bu_1t = df_bu_1t[df_bu_1t.NM_MUNICIPIO.eq('MANAUS')] \
                  .drop(columns=['DT_GERACAO',	'HH_GERACAO',	'CD_TIPO_ELEICAO', 'DT_BU_RECEBIDO',	'CD_TIPO_URNA', 'DS_TIPO_URNA',	
                                    'CD_TIPO_VOTAVEL',	'NR_URNA_EFETIVADA',	'CD_CARGA_1_URNA_EFETIVADA',	'CD_CARGA_2_URNA_EFETIVADA',	
                                    'CD_FLASHCARD_URNA_EFETIVADA',	'DT_CARGA_URNA_EFETIVADA',	'DS_CARGO_PERGUNTA_SECAO',	'DS_AGREGADAS',
                                    'DT_ABERTURA',	'DT_ENCERRAMENTO',	'QT_ELEITORES_BIOMETRIA_NH',	'NR_JUNTA_APURADORA',	'NR_TURMA_APURADORA'])
df_bu_1t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590874 entries, 0 to 848940
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO_ELEICAO        590874 non-null  int64 
 1   NM_TIPO_ELEICAO    590874 non-null  object
 2   CD_PLEITO          590874 non-null  int64 
 3   DT_PLEITO          590874 non-null  object
 4   NR_TURNO           590874 non-null  int64 
 5   CD_ELEICAO         590874 non-null  int64 
 6   DS_ELEICAO         590874 non-null  object
 7   SG_UF              590874 non-null  object
 8   CD_MUNICIPIO       590874 non-null  int64 
 9   NM_MUNICIPIO       590874 non-null  object
 10  NR_ZONA            590874 non-null  int64 
 11  NR_SECAO           590874 non-null  int64 
 12  NR_LOCAL_VOTACAO   590874 non-null  int64 
 13  CD_CARGO_PERGUNTA  590874 non-null  int64 
 14  DS_CARGO_PERGUNTA  590874 non-null  object
 15  NR_PARTIDO         590874 non-null  int64 
 16  SG_PARTIDO         5

### Candidatos a Prefeito

In [52]:
df_prefeitos = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Prefeito') & df_bu_1t.DS_TIPO_VOTAVEL.eq('Nominal')] \
                      [['CD_ELEICAO','DS_CARGO_PERGUNTA','NR_VOTAVEL','SG_PARTIDO','NM_VOTAVEL']] \
                         .drop_duplicates() \
                         .sort_values(['NR_VOTAVEL'])
df_prefeitos.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_prefeitos

,eleicao,cargo,numero,partido,candidato
6,426,Prefeito,10,REPUBLICANOS,CAPITÃO ALBERTO NETO
9,426,Prefeito,13,PT,ZE RICARDO
209,426,Prefeito,16,PSTU,GILBERTO VASCONCELOS
7,426,Prefeito,19,PODE,AMAZONINO MENDES
5,426,Prefeito,22,PL,ALFREDO NASCIMENTO
0,426,Prefeito,27,DC,CHICO PRETO
10,426,Prefeito,30,NOVO,ROMERO REIS
8,426,Prefeito,51,PATRIOTA,CORONEL MENEZES
4,426,Prefeito,55,PSD,RICARDO NICOLAU
217,426,Prefeito,65,PC do B,MARCELO AMIL


In [53]:
df_prefeitos.to_csv('/content/datasets/prefeitos.csv')

### Resultados para Prefeito

In [54]:
df_resultados_prefeito = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Prefeito')] \
    [['CD_ELEICAO',	'NR_ZONA'	,'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA'	,'NR_VOTAVEL',	'QT_VOTOS']] \
    .sort_values(['NR_ZONA','NR_SECAO','NR_VOTAVEL'])
df_resultados_prefeito.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultados_prefeito

,eleicao,zona,secao,local,cargo,numero,votos
6,426,1,640,2429,Prefeito,10,19
9,426,1,640,2429,Prefeito,13,49
7,426,1,640,2429,Prefeito,19,90
5,426,1,640,2429,Prefeito,22,7
0,426,1,640,2429,Prefeito,27,7
...,...,...,...,...,...,...,...
848806,426,70,808,1848,Prefeito,51,13
848799,426,70,808,1848,Prefeito,55,19
848804,426,70,808,1848,Prefeito,70,46
848796,426,70,808,1848,Prefeito,95,9


In [55]:
filename = '/content/datasets/resultados_prefeito.csv'
df_resultados_prefeito.to_csv(filename)
print(f'Resultado da votação para prefeito no 1o turno gravada em \'{filename}\'')

Resultado da votação para prefeito no 1o turno gravada em '/content/datasets/resultados_prefeito.csv'


### Candidatos a Vereador

In [56]:
df_vereadores = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Vereador') & df_bu_1t.DS_TIPO_VOTAVEL.eq('Nominal')] \
                        [['CD_ELEICAO','DS_CARGO_PERGUNTA','NR_VOTAVEL','SG_PARTIDO','NM_VOTAVEL']] \
                         .drop_duplicates() \
                         .sort_values(['NR_VOTAVEL'])
df_vereadores.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_vereadores


,eleicao,cargo,numero,partido,candidato
3083,426,Vereador,10000,REPUBLICANOS,CABO COUTINHO
32033,426,Vereador,10007,REPUBLICANOS,SARGENTO ANA CLAUDIA
36326,426,Vereador,10008,REPUBLICANOS,CHAY CHANTEL
4547,426,Vereador,10010,REPUBLICANOS,LUCAS SALES
453,426,Vereador,10018,REPUBLICANOS,JOSUE DOS CAMINHONEIROS
...,...,...,...,...,...
950,426,Vereador,90800,PROS,EMERSON BOTELHO
10173,426,Vereador,90888,PROS,LORA DO SOCIAL
1335,426,Vereador,90900,PROS,ERICO BRAGA
1615,426,Vereador,90901,PROS,LUIZ ALBERTO


In [57]:
df_vereadores.to_csv('/content/datasets/vereadores.csv')

### Resultados para Vereador

In [58]:
df_resultados_vereador = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Vereador')] \
    [['CD_ELEICAO',	'NR_ZONA'	,'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA'	,'NR_VOTAVEL',	'QT_VOTOS']] \
    .sort_values(['NR_ZONA','NR_SECAO','NR_VOTAVEL'])
df_resultados_vereador.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultados_vereador

,eleicao,zona,secao,local,cargo,numero,votos
200,426,1,640,2429,Vereador,11,1
104,426,1,640,2429,Vereador,13,2
185,426,1,640,2429,Vereador,18,1
25,426,1,640,2429,Vereador,19,2
54,426,1,640,2429,Vereador,50,1
...,...,...,...,...,...,...,...
848823,426,70,808,1848,Vereador,77888,4
848898,426,70,808,1848,Vereador,90177,2
848832,426,70,808,1848,Vereador,90321,1
848830,426,70,808,1848,Vereador,90523,1


In [59]:
filename = '/content/datasets/resultados_vereador.csv'
df_resultados_vereador.to_csv(filename)
print(f'Resultado da votação para Vereador no 1o turno gravada em \'{filename}\'')

Resultado da votação para Vereador no 1o turno gravada em '/content/datasets/resultados_vereador.csv'


### Candidatos a Prefeito (2o turno)

In [60]:
# boletins de urna do 2o turno
!unzip -u bweb_2t_AM_301120201245.zip -d /content/work

Archive:  bweb_2t_AM_301120201245.zip
  inflating: /content/work/bweb_2t_AM_301120201245.csv  


In [61]:
df_bu_2t = pd.read_csv('/content/work/bweb_2t_AM_301120201245.csv', encoding="ISO-8859-1", delimiter=';')
df_bu_2t.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DT_BU_RECEBIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,CD_TIPO_URNA,DS_TIPO_URNA,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,DT_ABERTURA,DT_ENCERRAMENTO,QT_ELEITORES_BIOMETRIA_NH,DT_EMISSAO_BU,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
1682,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,2,731,1872,11,Prefeito,-1,#NULO#,#NULO#,29/11/2020 18:13:28,356,223,133,1,APURADA,3,Nulo,96,Nulo,14,1789426,687.690.639.978.208.530.,521.995,36FCFA09,06/11/2020 08:51:00,11 - 731,#NULO#,29/11/2020 07:00:00,29/11/2020 17:00:46,0,29/11/2020 17:01:53,-1,-1
1137,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,2,583,1872,11,Prefeito,19,PODE,Podemos,29/11/2020 18:13:21,369,293,76,1,APURADA,1,Nominal,19,AMAZONINO MENDES,128,1181552,495.771.837.002.526.650.,936.795,381A20C9,06/11/2020 09:48:00,11 - 583,#NULO#,29/11/2020 07:00:00,29/11/2020 17:02:22,0,29/11/2020 17:03:15,-1,-1
3429,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,32,821,2127,11,Prefeito,-1,#NULO#,#NULO#,29/11/2020 18:24:00,376,279,97,1,APURADA,3,Nulo,96,Nulo,18,1179004,283.671.253.875.611.192.,742.685,B1F0C25F,09/11/2020 15:22:00,11 - 821,#NULO#,29/11/2020 07:01:35,29/11/2020 17:06:23,0,29/11/2020 17:07:37,-1,-1
1175,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,2,593,2020,11,Prefeito,19,PODE,Podemos,29/11/2020 18:13:44,345,254,91,1,APURADA,1,Nominal,19,AMAZONINO MENDES,114,1180377,859.307.265.062.885.837.,483.052,18D653ED,06/11/2020 09:48:00,11 - 593,#NULO#,29/11/2020 07:03:10,29/11/2020 17:03:57,0,29/11/2020 17:05:41,-1,-1
13185,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,70,730,1821,11,Prefeito,19,PODE,Podemos,29/11/2020 18:27:23,423,294,129,1,APURADA,1,Nominal,19,AMAZONINO MENDES,151,1789219,465.495.307.975.497.488.,648.789,6D01AF77,10/11/2020 15:19:00,11 - 730,391,29/11/2020 07:00:00,29/11/2020 17:03:08,0,29/11/2020 17:05:01,-1,-1


In [114]:
df_prefeitos_2t = df_bu_2t[df_bu_2t.DS_CARGO_PERGUNTA.eq('Prefeito') & df_bu_2t.DS_TIPO_VOTAVEL.eq('Nominal')] \
                       [['CD_ELEICAO','DS_CARGO_PERGUNTA','NR_VOTAVEL','SG_PARTIDO','NM_VOTAVEL']] \
                         .drop_duplicates() \
                         .sort_values(['NR_VOTAVEL'])
df_prefeitos_2t.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_prefeitos_2t

,eleicao,cargo,numero,partido,candidato
1,427,Prefeito,19,PODE,AMAZONINO MENDES
0,427,Prefeito,70,AVANTE,DAVID ALMEIDA


In [115]:
filename = '/content/datasets/prefeitos_2t.csv'
df_prefeitos_2t.to_csv(filename)
print(f'Candidatos a Prefeito (2o turno) gravados em \'{filename}\'')

Candidatos a Prefeito (2o turno) gravados em '/content/datasets/prefeitos_2t.csv'


### Resultados para Prefeito (2o turno)

In [64]:
df_resultados_prefeito_2t = df_bu_2t[df_bu_2t.DS_CARGO_PERGUNTA.eq('Prefeito')] \
    [['CD_ELEICAO',	'NR_ZONA'	,'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA'	,'NR_VOTAVEL',	'QT_VOTOS']] \
    .sort_values(['NR_ZONA','NR_SECAO','NR_VOTAVEL'])
df_resultados_prefeito_2t.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultados_prefeito_2t

,eleicao,zona,secao,local,cargo,numero,votos
1,427,1,640,2429,Prefeito,19,156
0,427,1,640,2429,Prefeito,70,135
3,427,1,640,2429,Prefeito,95,10
2,427,1,640,2429,Prefeito,96,29
4,427,1,641,2089,Prefeito,19,153
...,...,...,...,...,...,...,...
13471,427,70,806,2089,Prefeito,96,16
13476,427,70,808,1848,Prefeito,19,68
13473,427,70,808,1848,Prefeito,70,122
13475,427,70,808,1848,Prefeito,95,11


In [65]:
filename = '/content/datasets/resultados_prefeito_2t.csv'
df_resultados_prefeito_2t.to_csv(filename)
print(f'Resultado da votação para Prefeito (2o turno) gravado em \'{filename}\'')

Resultado da votação para Prefeito (2o turno) gravado em '/content/datasets/resultados_prefeito_2t.csv'


### Eleições

In [66]:
df1 = df_bu_1t[['ANO_ELEICAO',	'NM_TIPO_ELEICAO',	'CD_PLEITO',	'DT_PLEITO', 'NR_TURNO',	'CD_ELEICAO',	'DS_ELEICAO']] \
               .drop_duplicates()
df2 = df_bu_2t[['ANO_ELEICAO',	'NM_TIPO_ELEICAO',	'CD_PLEITO',	'DT_PLEITO', 'NR_TURNO',	'CD_ELEICAO',	'DS_ELEICAO']] \
               .drop_duplicates()
df_eleicao = df1.append(df2)
df_eleicao.columns = ['ano', 'tipo', 'pleito', 'data', 'turno', 'eleicao', 'descricao']
df_eleicao

,ano,tipo,pleito,data,turno,eleicao,descricao
0,2020,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020
0,2020,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020


In [67]:
filename = '/content/datasets/eleicoes.csv'
df_eleicao.to_csv(filename)
print(f'Eleições gravadas em \'{filename}\'')

Eleições gravadas em '/content/datasets/eleicoes.csv'


### Limpeza do diretório de trabalho

In [68]:
# Limpeza do diretório temporário de trabalho
!rm -rf /content/work/*

## Locais de votação

In [69]:
# detalhes das seções
!unzip -u detalhe_votacao_secao_2020.zip -d /content/work

Archive:  detalhe_votacao_secao_2020.zip
  inflating: /content/work/detalhe_votacao_secao_2020_AC.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AL.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AM.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AP.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_BA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_BRASIL.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_CE.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_ES.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_GO.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MG.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MS.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MT.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_PA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_PB.csv  
  inflatin

In [70]:
df_dvs = pd.read_csv('/content/work/detalhe_votacao_secao_2020_BRASIL.csv', encoding="ISO-8859-1", delimiter=';')
df_dvs.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA,QT_VOTOS_ANULADOS_APU_SEP,NR_LOCAL_VOTACAO
816574,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,CE,15598,SOBRAL,15598,SOBRAL,121,9,11,PREFEITO,371,314,57,293,8,13,0,0,1341
786898,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,PI,12190,TERESINA,12190,TERESINA,97,98,11,PREFEITO,330,264,66,247,5,12,0,0,1287
480949,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,RS,85111,ALVORADA,85111,ALVORADA,74,339,13,VEREADOR,414,286,128,244,20,13,9,0,1180
138738,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,MG,48518,MONSENHOR PAULO,48518,MONSENHOR PAULO,61,29,13,VEREADOR,343,274,69,241,2,8,23,0,1040
533849,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,SP,71072,SÃO PAULO,71072,SÃO PAULO,349,177,13,VEREADOR,379,228,151,157,21,28,22,0,1139


In [71]:
df_dvs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897536 entries, 0 to 897535
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   DT_GERACAO                 897536 non-null  object
 1   HH_GERACAO                 897536 non-null  object
 2   ANO_ELEICAO                897536 non-null  int64 
 3   CD_TIPO_ELEICAO            897536 non-null  int64 
 4   NM_TIPO_ELEICAO            897536 non-null  object
 5   NR_TURNO                   897536 non-null  int64 
 6   CD_ELEICAO                 897536 non-null  int64 
 7   DS_ELEICAO                 897536 non-null  object
 8   DT_ELEICAO                 897536 non-null  object
 9   TP_ABRANGENCIA             897536 non-null  object
 10  SG_UF                      897536 non-null  object
 11  SG_UE                      897536 non-null  int64 
 12  NM_UE                      897536 non-null  object
 13  CD_MUNICIPIO               897536 non-null  

### Seções

In [72]:
df_secoes = df_dvs[df_dvs.NM_MUNICIPIO.eq('MANAUS')] \
                  [['CD_ELEICAO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
                    'QT_APTOS',	'QT_COMPARECIMENTO',	'QT_ABSTENCOES']] \
                    .sort_values(['NR_ZONA', 'NR_SECAO', 'CD_ELEICAO']) \
                    .drop_duplicates()
df_secoes.columns = ['eleicao', 'zona', 'secao', 'local','aptos','votos','abstencoes']
df_secoes

,eleicao,zona,secao,local,aptos,votos,abstencoes
41373,426,1,640,2429,411,348,63
725917,427,1,640,2429,411,330,81
325496,426,1,641,2089,393,326,67
328669,427,1,641,2089,393,308,85
138660,426,1,642,2321,397,341,56
...,...,...,...,...,...,...,...
431441,427,70,805,2046,366,245,121
291560,426,70,806,2089,399,280,119
604720,427,70,806,2089,399,277,122
374725,426,70,808,1848,284,224,60


In [73]:
filename = '/content/datasets/secoes.csv'
df_secoes.to_csv(filename)
print(f'Seções eleitorais gravadas em \'{filename}\'')

Seções eleitorais gravadas em '/content/datasets/secoes.csv'


In [74]:
dft_dvs = df_dvs[df_dvs.NM_MUNICIPIO.eq('MANAUS')] \
               [['CD_ELEICAO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO',
                 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES',
                 'QT_VOTOS_NOMINAIS',	'QT_VOTOS_BRANCOS',	'QT_VOTOS_NULOS', 'QT_VOTOS_LEGENDA']] \
                .sort_values(['CD_ELEICAO','NR_ZONA', 'NR_SECAO'])
dft_dvs

,CD_ELEICAO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA
41373,426,1,640,2429,PREFEITO,411,348,63,314,15,19,0
205055,426,1,640,2429,VEREADOR,411,348,63,302,19,16,11
325496,426,1,641,2089,PREFEITO,393,326,67,293,10,23,0
340194,426,1,641,2089,VEREADOR,393,326,67,292,11,13,10
138660,426,1,642,2321,VEREADOR,397,341,56,306,12,17,6
...,...,...,...,...,...,...,...,...,...,...,...,...
891487,427,70,803,2003,PREFEITO,275,188,87,170,12,6,0
624649,427,70,804,2038,PREFEITO,332,210,122,183,12,15,0
431441,427,70,805,2046,PREFEITO,366,245,121,216,13,16,0
604720,427,70,806,2089,PREFEITO,399,277,122,242,19,16,0


### Escolas

In [75]:
# detalhes das seções
!unzip -u eleitorado_local_votacao_2020.zip -d /content/work

Archive:  eleitorado_local_votacao_2020.zip
  inflating: /content/work/LEIAME.pdf  
  inflating: /content/work/eleitorado_local_votacao_2020.csv  


In [76]:
df = pd.read_csv('/content/work/eleitorado_local_votacao_2020.csv', encoding="ISO-8859-1", delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597429 entries, 0 to 597428
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_GERACAO              597429 non-null  object 
 1   HH_GERACAO              597429 non-null  object 
 2   AA_ELEICAO              597429 non-null  int64  
 3   DT_ELEICAO              597429 non-null  object 
 4   DS_ELEICAO              597429 non-null  object 
 5   SG_UF                   597429 non-null  object 
 6   CD_MUNICIPIO            597429 non-null  int64  
 7   NM_MUNICIPIO            597429 non-null  object 
 8   NR_ZONA                 597429 non-null  int64  
 9   NR_SECAO                597429 non-null  int64  
 10  CD_TIPO_SECAO_AGREGADA  597429 non-null  int64  
 11  DS_TIPO_SECAO_AGREGADA  597429 non-null  object 
 12  NR_LOCAL_VOTACAO        597429 non-null  int64  
 13  NM_LOCAL_VOTACAO        597429 non-null  object 
 14  CD_TIPO_LOCAL       

In [77]:
df[df.NR_ZONA.eq(32) & df.NM_MUNICIPIO.eq('MANAUS')]

,DT_GERACAO,HH_GERACAO,AA_ELEICAO,DT_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_SECAO_AGREGADA,DS_TIPO_SECAO_AGREGADA,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,CD_TIPO_LOCAL,DS_TIPO_LOCAL,DS_ENDERECO,NM_BAIRRO,NR_CEP,NR_TELEFONE_LOCAL,NR_LATITUDE,NR_LONGITUDE,CD_SITU_LOCAL_VOTACAO,DS_SITU_LOCAL_VOTACAO,CD_SITU_ZONA,DS_SITU_ZONA,CD_SITU_SECAO,DS_SITU_SECAO,CD_SITU_LOCALIDADE,DS_SITU_LOCALIDADE,QT_ELEITOR,QT_ELEITOR_ELEICAO
225,27/11/2020,13:04:41,2020,29/11/2020,2º Turno,AM,2550,MANAUS,32,787,1,Principal,2046,E. E. BENJAMIN MAGALHÃES BRANDÃO,1,Convencional,"RUA BELO HORIZONTE, S/N",COMPENSA,69036100,-1,-3.098930,-60.062220,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,378,378
1911,27/11/2020,13:04:41,2020,15/11/2020,1º Turno,AM,2550,MANAUS,32,647,1,Principal,2135,E. M. MARIA FERNANDES,1,Convencional,"RUA DA PROSPERIDADE, S/N",COMPENSA,69035640,-1,-3.104619,-60.061595,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,346,349
2548,27/11/2020,13:04:41,2020,15/11/2020,1º Turno,AM,2550,MANAUS,32,707,1,Principal,1937,E. E. SANTO ANTONIO,1,Convencional,RUA LAURO BITENCOURT,SANTO ANTONIO,69029060,36713031,-3.116130,-60.042020,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,345,344
3146,27/11/2020,13:04:41,2020,29/11/2020,2º Turno,AM,2550,MANAUS,32,704,1,Principal,1970,E. E. GENERAL SAMPAIO,1,Convencional,RUA BENJAMIN LIMA (PROX. AO FUETH) S/N,SAO JORGE,69033640,32168447,-3.110135,-60.039716,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,350,347
4488,27/11/2020,13:04:41,2020,29/11/2020,2º Turno,AM,2550,MANAUS,32,682,1,Principal,2143,E. E. JULIA BITTENCOURT,1,Convencional,"AV. BRASIL, 2845",COMPENSA,69036110,-1,-3.108960,-60.051756,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,380,380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590279,27/11/2020,13:04:41,2020,15/11/2020,1º Turno,AM,2550,MANAUS,32,942,3,Distribuída de ofício,2097,E. M. ELVIRA BORGES,1,Convencional,"RUA DA PROSPERIDADE, S/N",COMPENSA,69035640,-1,-3.105272,-60.061882,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,394,0
592310,27/11/2020,13:04:41,2020,15/11/2020,1º Turno,AM,2550,MANAUS,32,671,1,Principal,2178,E. M. SÃO PEDRO,1,Convencional,"RUA MATOS AREOSA, S/N",COMPENSA,69035100,-1,-3.108448,-60.057293,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,357,490
595554,27/11/2020,13:04:41,2020,15/11/2020,1º Turno,AM,2550,MANAUS,32,841,1,Principal,2003,E. M. PAULA FRASSINETTI,1,Convencional,"AV. PRES. DUTRA, S/N",GLORIA,69027130,36736839,-3.119693,-60.036436,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,295,291
595772,27/11/2020,13:04:41,2020,29/11/2020,2º Turno,AM,2550,MANAUS,32,678,1,Principal,2267,E. E. PEDRO SILVESTRE,1,Convencional,RUA RIO BRANCO,SAO RAIMUNDO,69027030,36171277,-3.127156,-60.040100,1,ATIVO,1,ATIVO,1,ATIVO,1,ATIVO,367,365


In [78]:
df_escolas = df[df.NM_MUNICIPIO.eq('MANAUS') & df.DS_TIPO_SECAO_AGREGADA.eq('Principal')] \
                  [['NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'NM_LOCAL_VOTACAO',
                    'NM_BAIRRO','NR_LATITUDE',	'NR_LONGITUDE']] \
                    .sort_values(['NR_ZONA', 'NR_SECAO']) \
                    .drop_duplicates()
df_escolas.columns = ['zona', 'secao', 'local', 'nome', 'bairro', 'lat','long']
df_escolas

,zona,secao,local,nome,bairro,lat,long
90957,1,640,2429,ESCOLA ESTADUAL CARVALHO LEAL (RECEBEU AS SEÇÕ...,CACHOEIRINHA,-1.000000,-1.000000
70444,1,641,2089,ESCOLA SUPERIOR DE CIENCIAS DA SAUDE - UEA,CACHOEIRINHA,-3.119252,-60.006131
179658,1,642,2321,E. E. MILBURGES BEZERRA DE ARAUJO,RAIZ,-3.123326,-59.995204
139511,1,643,2054,E. E. 1º DE MAIO,PRACA 14 DE JANEIRO,-3.118991,-60.011749
494185,1,644,2402,E. E. SOLON DE LUCENA,SAO GERALDO,-3.104937,-60.027054
...,...,...,...,...,...,...,...
142013,70,803,2003,E. M. ALMERON CAMINHA MONTEIRO,CIDADE DE DEUS,-3.020089,-59.950070
382031,70,804,2038,CMEI PADRE CLAUDIO DALBON,NOVO ALEIXO,-3.043992,-59.957957
509896,70,805,2046,CMEI TEN ROXANA PEREIRA BONESSI,NOVO ALEIXO,-3.036308,-59.942780
216887,70,806,2089,E. M. BIÓLOGO ADOLPHO DUCKE,CIDADE DE DEUS,-3.018721,-59.939805


In [79]:
filename = '/content/datasets/escolas.csv'
df_escolas.to_csv(filename)
print(f'Escolas gravadas em \'{filename}\'')

Escolas gravadas em '/content/datasets/escolas.csv'


In [80]:
# Limpeza do diretório temporário de trabalho
!rm -rf /content/work/*

## Dataset de Seções

| Atributo | Descrição |
| --- | --- |
| ZONA | Zona eleitoral |
| SECAO | Seção eleitoral |
| ESCOLA | Escola onde a seção eleitoral está situada |
| BAIRRO | Bairro onde a escola está localizada
| LAT | Latitude do ponto onde a escola está localizada |
| LONG | Longitude od ponto onde a escola está localizada |
| APTOS | Quantidade de eleitores aptos a votar |
| VOTOS | Quantidade de eleitores que votou (1o turno) |
| ABSTENCOES | Quantidade de eleitores que não compareceu (1o turno) |
| VOTOS_2T | Quantidade de eleitores que votou (2o turno) |
| ABTENCOES_2T | Quantidade de eleitores que não compareceu (2o turno) |
| PREF_1ST | Nome do candidato a prefeito que ficou em primeiro lugar (1o turno) |
| PREF_1ST_PART | Partido do candidato a prefeito que ficou em primeiro lugar (1o turno) |
| PREF_1ST_VOTOS | Votos computados para o candidato a prefeito que ficou em primeiro lugar (1o turno) |
| PREF_2ND | Nome do candidato a prefeito que ficou em segundo lugar (1o turno) |
| PREF_2ND_PART | Partido do candidato a prefeito que ficou em segundo lugar (1o turno) |
| PREF_2ND_VOTOS | Votos computados para o candidato a prefeito que ficou em segundo lugar (1o turno) |
| PREF_3RD | Nome do candidato a prefeito que ficou em terceiro lugar (1o turno) |
| PREF_3RD_PART | Partido do candidato a prefeito que ficou em terceiro lugar (1o turno) |
| PREF_3RD_VOTOS | Votos computados para o candidato a prefeito que ficou em terceiro lugar (1o turno) |
| PREF_4TH | Nome do candidato a prefeito que ficou em quarto lugar (1o turno) |
| PREF_4TH_PART | Partido do candidato a prefeito que ficou em quarto lugar (1o turno) |
| PREF_4TH_VOTOS | Votos computados para o candidato a prefeito que ficou em quarto lugar (1o turno) |
| PART_1ST_VER | Partido com maior quantidade de votos para vereador (1o turno) |

---
```
#locais
df_secoes.columns = ['eleicao', 'zona', 'secao', 'local', 'aptos','votos','abstencoes']
df_escolas.columns = ['zona', 'secao', 'local', 'nome', 'bairro', 'lat','long']

#candidatos
df_prefeitos.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_vereadores.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_prefeitos_2t.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']

#resultados
df_resultados_prefeito.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos'] 
df_resultados_vereador.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultado_prefeito.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
```

In [81]:
df_secoes_final = df_escolas.copy()
df_secoes_final.sample(5)

,zona,secao,local,nome,bairro,lat,long
398271,37,639,1996,E. M. AMINE DAOU LINDOSO,JAPIIM,-3.117860,-59.991109
339551,63,689,1732,E. M. ARMANDO MENDES,SAO JOSE OPERARIO,-3.067364,-59.944393
221032,40,486,1457,E. E. SEN PETRONIO PORTELLA,DOM PEDRO,-3.084008,-60.046178
188091,59,564,2313,E. E. PROF. MARIA TEIXEIRA GOES,ZUMBI DOS PALMARES,-3.081828,-59.944452
108830,2,675,1902,ESCOLA SUPERIOR DE TECNOLOGIA - UEA,PARQUE 10 DE NOVEMBRO,-3.091319,-60.018083


### Adicionando quantitativos de eleitores

In [82]:
def create_or_replace_column(df, col, val):
  if col in df.columns:
    del df[col]

  df[col] = val

create_or_replace_column(df_secoes_final, 'aptos', 0)
create_or_replace_column(df_secoes_final, 'votos', 0)
create_or_replace_column(df_secoes_final, 'abstencoes', 0)
create_or_replace_column(df_secoes_final, 'votos_2t', 0)
create_or_replace_column(df_secoes_final, 'abstencoes_2t', 0)
df_secoes_final.columns

Index(['zona', 'secao', 'local', 'nome', 'bairro', 'lat', 'long', 'aptos',
       'votos', 'abstencoes', 'votos_2t', 'abstencoes_2t'],
      dtype='object')

In [83]:
df = df_secoes.groupby(['zona','secao', 'eleicao'])[['aptos','votos','abstencoes']].first().unstack()
df.head()

aptos      votos      abstencoes    
eleicao      426  427   426  427        426 427
zona secao                                     
1    640     411  411   348  330         63  81
     641     393  393   326  308         67  85
     642     397  397   341  332         56  65
     643     480  480   414  398         66  82
     644     328  328   277  265         51  63

In [84]:
def get_total_aptos(row):
  return df.loc[row['zona'], row['secao']].loc['aptos'].loc[426]

def get_total_votos(row):
  return df.loc[row['zona'], row['secao']].loc['votos'].loc[426]

def get_total_abstencoes(row):
  return df.loc[row['zona'], row['secao']].loc['abstencoes'].loc[426]
  
df_secoes_final['aptos'] = df_secoes_final.apply(get_total_aptos, axis=1)
df_secoes_final['votos'] = df_secoes_final.apply(get_total_votos, axis=1)
df_secoes_final['abstencoes'] = df_secoes_final.apply(get_total_abstencoes, axis=1)
df_secoes_final.sample(4)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t
87226,63,608,1813,IFAM (COLÉGIO AGRÍCOLA),GILBERTO MESTRINHO,-3.080023,-59.933888,435,381,54,0,0
108826,31,384,1899,E. E ANTONIO LUCENA BITENCOURT,BETANIA,-3.136398,-59.999446,394,318,76,0,0
346281,59,540,2178,E. M. ETELVINA PEREIRA BRAGA,COROADO,-3.082229,-59.968872,426,378,48,0,0
82930,59,624,2194,E. M. JOÃO DOS SANTOS BRAGA,ZUMBI DOS PALMARES,-3.074387,-59.944248,372,333,39,0,0


In [85]:
def get_total_votos_2t(row):
  return df.loc[row['zona'], row['secao']].loc['votos'].loc[427]

def get_total_abstencoes_2t(row):
  return df.loc[row['zona'], row['secao']].loc['abstencoes'].loc[427]
  
df_secoes_final['votos_2t'] = df_secoes_final.apply(get_total_votos_2t, axis=1)
df_secoes_final['abstencoes_2t'] = df_secoes_final.apply(get_total_abstencoes_2t, axis=1)
df_secoes_final.sample(4)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t
181045,59,723,2194,E. M. JOÃO DOS SANTOS BRAGA,ZUMBI DOS PALMARES,-3.074387,-59.944248,361,285,76,243,118
376295,32,696,2054,E. M. CARLOS GOMES,COMPENSA,-3.096741,-60.050366,392,324,68,311,81
342617,2,768,1910,UNIP,PARQUE 10 DE NOVEMBRO,-3.080580,-60.020971,381,278,103,247,134
308665,63,539,2003,E. E. PROF. JORGE KARAM NETO,TANCREDO NEVES,-3.055777,-59.948282,413,347,66,331,82


### Adicionando o resultado da eleição para prefeito

In [86]:
df = df_resultados_prefeito.merge(df_prefeitos, on=['eleicao', 'cargo', 'numero'])
df.head()

,eleicao,zona,secao,local,cargo,numero,votos,partido,candidato
0,426,1,640,2429,Prefeito,10,19,REPUBLICANOS,CAPITÃO ALBERTO NETO
1,426,1,641,2089,Prefeito,10,9,REPUBLICANOS,CAPITÃO ALBERTO NETO
2,426,1,642,2321,Prefeito,10,28,REPUBLICANOS,CAPITÃO ALBERTO NETO
3,426,1,643,2054,Prefeito,10,24,REPUBLICANOS,CAPITÃO ALBERTO NETO
4,426,1,644,2402,Prefeito,10,21,REPUBLICANOS,CAPITÃO ALBERTO NETO


In [87]:
def get_nth_prefeito_mais_votado(df, nth):
  return df[['zona', 'secao', 'votos', 'candidato', 'partido']] \
              .sort_values('votos', ascending=False) \
              .groupby(['zona', 'secao']).nth(nth) \
              .reset_index()

In [88]:
# define o primeiro prefeito mais votado em cada seção (1o turno)
df_pref_1st = get_nth_prefeito_mais_votado(df, 0)
df_pref_1st.columns = ['zona', 'secao', 'votos_1st', 'candidato_1st', 'partido_1st']
df_pref_1st.sample(5)

,zona,secao,votos_1st,candidato_1st,partido_1st
1765,59,500,110,DAVID ALMEIDA,AVANTE
3059,68,655,89,DAVID ALMEIDA,AVANTE
2731,65,463,51,DAVID ALMEIDA,AVANTE
1364,40,493,74,DAVID ALMEIDA,AVANTE
2149,62,448,73,AMAZONINO MENDES,PODE


In [89]:
df_secoes_final = df_secoes_final.merge(df_pref_1st, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st
367,2,670,1856,FACULDADE ESTÁCIO (ANTIGO CEL),CHAPADA,-3.091966,-60.027744,371,278,93,283,88,69,AMAZONINO MENDES,PODE
2700,65,432,1627,E. M. JORNALISTA SABÁ RAPOSO,MONTE DAS OLIVEIRAS,-3.019994,-59.994799,420,353,67,345,75,73,AMAZONINO MENDES,PODE
3037,68,632,1708,E. M. PROF. ALVARO VALLE,JORGE TEIXEIRA,-3.035854,-59.934505,386,319,67,306,80,85,DAVID ALMEIDA,AVANTE


In [90]:
# define o segundo prefeito mais votado em cada seção (1o turno)
df_pref_2nd = get_nth_prefeito_mais_votado(df, 1)
df_pref_2nd.columns = ['zona', 'secao', 'votos_2nd', 'candidato_2nd', 'partido_2nd']
df_pref_2nd.sample(5)

,zona,secao,votos_2nd,candidato_2nd,partido_2nd
2042,62,341,60,DAVID ALMEIDA,AVANTE
1165,37,707,39,AMAZONINO MENDES,PODE
809,32,770,68,AMAZONINO MENDES,PODE
2897,68,488,64,AMAZONINO MENDES,PODE
727,32,686,83,DAVID ALMEIDA,AVANTE


In [91]:
df_secoes_final = df_secoes_final.merge(df_pref_2nd, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd
2948,68,543,1660,E. M. IRMA HELENA AUGUSTA WALCOT,JORGE TEIXEIRA,-3.045025,-59.925176,385,324,61,309,76,71,DAVID ALMEIDA,AVANTE,69,AMAZONINO MENDES,PODE
502,31,589,2151,E. E. ESTELITA TAPAJÓS,EDUCANDOS,-3.142238,-60.014893,371,340,31,328,43,95,DAVID ALMEIDA,AVANTE,74,AMAZONINO MENDES,PODE
849,32,812,2267,E. E. PEDRO SILVESTRE,SAO RAIMUNDO,-3.127156,-60.040100,488,403,85,386,102,108,AMAZONINO MENDES,PODE,62,DAVID ALMEIDA,AVANTE


In [92]:
# define o terceiro prefeito mais votado em cada seção (1o turno)
df_pref_3rd = get_nth_prefeito_mais_votado(df, 2)
df_pref_3rd.columns = ['zona', 'secao', 'votos_3rd', 'candidato_3rd', 'partido_3rd']
df_pref_3rd.sample(5)

,zona,secao,votos_3rd,candidato_3rd,partido_3rd
3255,70,687,39,ZE RICARDO,PT
709,32,668,46,RICARDO NICOLAU,PSD
370,2,673,42,CORONEL MENEZES,PATRIOTA
92,1,740,61,RICARDO NICOLAU,PSD
1212,40,341,34,ZE RICARDO,PT


In [93]:
df_secoes_final = df_secoes_final.merge(df_pref_3rd, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd
2135,62,434,1660,E. E. PROF. WALDOCKE FRICK DE LIRA- COLÉGIO MI...,TARUMA,-3.003390,-60.033918,407,337,70,324,83,62,AMAZONINO MENDES,PODE,58,DAVID ALMEIDA,AVANTE,53,RICARDO NICOLAU,PSD
1764,59,499,2240,E. E. ARISTOTELES COMTE DE ALENCAR,COROADO,-3.093048,-59.981018,373,327,46,324,49,79,AMAZONINO MENDES,PODE,48,DAVID ALMEIDA,AVANTE,44,RICARDO NICOLAU,PSD
2181,62,480,1678,E. E. TEREZA DE JESUS VASCONCELOS DIAS,REDENCAO,-3.060276,-60.037988,401,313,88,304,97,72,AMAZONINO MENDES,PODE,57,DAVID ALMEIDA,AVANTE,41,RICARDO NICOLAU,PSD


In [94]:
# define o quarto prefeito mais votado em cada seção (1o turno)
df_pref_4th = get_nth_prefeito_mais_votado(df, 3)
df_pref_4th.columns = ['zona', 'secao', 'votos_4th', 'candidato_4th', 'partido_4th']
df_pref_4th.sample(5)

,zona,secao,votos_4th,candidato_4th,partido_4th
1006,37,546,43,ZE RICARDO,PT
1437,40,566,36,RICARDO NICOLAU,PSD
274,2,572,38,CORONEL MENEZES,PATRIOTA
448,2,769,35,ZE RICARDO,PT
3027,68,623,40,RICARDO NICOLAU,PSD


In [95]:
df_secoes_final = df_secoes_final.merge(df_pref_4th, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th
707,32,666,2097,E. M. ELVIRA BORGES,COMPENSA,-3.105272,-60.061882,391,333,58,319,72,82,DAVID ALMEIDA,AVANTE,67,AMAZONINO MENDES,PODE,58,ZE RICARDO,PT,44,RICARDO NICOLAU,PSD
286,2,586,2046,CENTRO UNIVERSITÁRIO NILTON LINS,FLORES,-3.058218,-60.006380,374,306,68,296,78,53,DAVID ALMEIDA,AVANTE,52,AMAZONINO MENDES,PODE,47,CORONEL MENEZES,PATRIOTA,37,RICARDO NICOLAU,PSD
2019,62,317,1619,E. E. ITACYARA MARQUES PINHO,PLANALTO,-3.073372,-60.061001,416,342,74,337,79,86,AMAZONINO MENDES,PODE,53,DAVID ALMEIDA,AVANTE,51,ZE RICARDO,PT,50,CORONEL MENEZES,PATRIOTA


### Adicionando o resultado da eleição para vereador

In [96]:
df = df_resultados_vereador.merge(df_vereadores, on=['eleicao', 'cargo', 'numero'])
df.head()

,eleicao,zona,secao,local,cargo,numero,votos,partido,candidato
0,426,1,640,2429,Vereador,10100,3,REPUBLICANOS,SERGINHO CAMARA
1,426,1,641,2089,Vereador,10100,3,REPUBLICANOS,SERGINHO CAMARA
2,426,1,642,2321,Vereador,10100,1,REPUBLICANOS,SERGINHO CAMARA
3,426,1,643,2054,Vereador,10100,2,REPUBLICANOS,SERGINHO CAMARA
4,426,1,645,2267,Vereador,10100,3,REPUBLICANOS,SERGINHO CAMARA


In [97]:
def get_nth_partido_mais_votado_vereador(df, nth):
  return df[['zona','secao','votos','partido']] \
          .groupby(['zona','secao','partido']).sum('votos') \
          .sort_values(['zona','secao','votos'], ascending=[True,True,False]).reset_index() \
          .groupby(['zona', 'secao']).nth(nth).reset_index()

In [98]:
# define o primeiro partido mais votado na eleição para vereador em cada seção (1o turno)
df_part_1st_ver = get_nth_partido_mais_votado_vereador(df, 0)
df_part_1st_ver.columns = ['zona', 'secao', 'partido_1st_ver', 'votos_1st_ver']
df_part_1st_ver.sample(5)

,zona,secao,partido_1st_ver,votos_1st_ver
2602,63,773,AVANTE,45
1650,58,923,PSB,21
3233,70,665,PMN,27
379,2,685,AVANTE,20
3205,70,637,PSDB,86


In [99]:
df_secoes_final = df_secoes_final.merge(df_part_1st_ver, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th,partido_1st_ver,votos_1st_ver
2800,65,532,1708,E. E. ERNESTO PINHO FILHO,NOVA CIDADE,-3.017055,-59.990963,396,310,86,275,121,57,AMAZONINO MENDES,PODE,46,RICARDO NICOLAU,PSD,46,DAVID ALMEIDA,AVANTE,45,ZE RICARDO,PT,PATRIOTA,23
1112,37,652,2011,E. E. BENICIO LEAO,JAPIIM,-3.107966,-59.983861,355,299,56,283,72,73,AMAZONINO MENDES,PODE,56,DAVID ALMEIDA,AVANTE,42,RICARDO NICOLAU,PSD,34,CORONEL MENEZES,PATRIOTA,CIDADANIA,35
447,2,768,1910,UNIP,PARQUE 10 DE NOVEMBRO,-3.080580,-60.020971,381,278,103,247,134,60,DAVID ALMEIDA,AVANTE,50,AMAZONINO MENDES,PODE,33,ZE RICARDO,PT,33,RICARDO NICOLAU,PSD,AVANTE,23


In [100]:
# define o segundo partido mais votado na eleição para vereador em cada seção (1o turno)
df_part_2nd_ver = get_nth_partido_mais_votado_vereador(df, 0)
df_part_2nd_ver.columns = ['zona', 'secao', 'partido_2nd_ver', 'votos_2nd_ver']
df_part_2nd_ver.sample(5)

,zona,secao,partido_2nd_ver,votos_2nd_ver
1554,58,814,AVANTE,29
926,32,902,PRTB,32
2883,65,626,CIDADANIA,27
1907,59,642,PSC,36
2824,65,558,PMN,21


In [101]:
df_secoes_final = df_secoes_final.merge(df_part_2nd_ver, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th,partido_1st_ver,votos_1st_ver,partido_2nd_ver,votos_2nd_ver
3266,70,699,1961,E. E. RAIMUNDA HOLANDA DE SOUZA,NOVO ALEIXO,-3.039003,-59.942638,386,323,63,315,71,81,DAVID ALMEIDA,AVANTE,53,AMAZONINO MENDES,PODE,50,ZE RICARDO,PT,38,RICARDO NICOLAU,PSD,AVANTE,27,AVANTE,27
2540,63,706,1929,E. M. ALBÉRICO ANTUNES DE OLIVEIRA,SAO JOSE OPERARIO,-3.069725,-59.939132,398,328,70,289,109,81,DAVID ALMEIDA,AVANTE,62,AMAZONINO MENDES,PODE,37,RICARDO NICOLAU,PSD,36,ZE RICARDO,PT,AVANTE,36,AVANTE,36
2914,68,508,1872,E. M. SÃO FRANCISCO,ZONA RURAL,-3.087752,-59.569404,130,97,33,93,37,24,AMAZONINO MENDES,PODE,19,DAVID ALMEIDA,AVANTE,13,ZE RICARDO,PT,12,RICARDO NICOLAU,PSD,PODE,24,PODE,24


In [102]:
# define o terceiro partido mais votado na eleição para vereador em cada seção (1o turno)
df_part_3rd_ver = get_nth_partido_mais_votado_vereador(df, 0)
df_part_3rd_ver.columns = ['zona', 'secao', 'partido_3rd_ver', 'votos_3rd_ver']
df_part_3rd_ver.sample(5)

,zona,secao,partido_3rd_ver,votos_3rd_ver
436,2,747,AVANTE,28
619,31,707,PATRIOTA,52
1492,58,741,PL,32
561,31,648,AVANTE,23
2619,65,268,PMN,26


In [103]:
df_secoes_final = df_secoes_final.merge(df_part_3rd_ver, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th,partido_1st_ver,votos_1st_ver,partido_2nd_ver,votos_2nd_ver,partido_3rd_ver,votos_3rd_ver
294,2,594,1929,E. E. ALICE SALERNO,PARQUE 10 DE NOVEMBRO,-3.086087,-60.014996,526,432,94,419,107,97,AMAZONINO MENDES,PODE,71,DAVID ALMEIDA,AVANTE,60,ZE RICARDO,PT,48,RICARDO NICOLAU,PSD,AVANTE,32,AVANTE,32,AVANTE,32
1027,37,567,1791,E. E. TIRADENTES,PETROPOLIS,-3.116420,-59.997239,380,341,39,331,49,82,DAVID ALMEIDA,AVANTE,69,AMAZONINO MENDES,PODE,40,ZE RICARDO,PT,34,RICARDO NICOLAU,PSD,REPUBLICANOS,35,REPUBLICANOS,35,REPUBLICANOS,35
2119,62,418,1813,E.M. JOAO GOULART,SANTA ETELVINA,-2.983991,-60.012382,397,348,49,326,71,94,AMAZONINO MENDES,PODE,54,DAVID ALMEIDA,AVANTE,52,ZE RICARDO,PT,41,RICARDO NICOLAU,PSD,PSC,37,PSC,37,PSC,37


In [104]:
# define o segundo partido mais votado na eleição para vereador em cada seção (1o turno)
df_part_4th_ver = get_nth_partido_mais_votado_vereador(df, 0)
df_part_4th_ver.columns = ['zona', 'secao', 'partido_4th_ver', 'votos_4th_ver']
df_part_4th_ver.sample(5)

,zona,secao,partido_4th_ver,votos_4th_ver
1139,37,679,PSDB,49
328,2,630,AVANTE,21
1690,58,971,PSC,26
64,1,710,AVANTE,17
1570,58,832,AVANTE,30


In [105]:
df_secoes_final = df_secoes_final.merge(df_part_4th_ver, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th,partido_1st_ver,votos_1st_ver,partido_2nd_ver,votos_2nd_ver,partido_3rd_ver,votos_3rd_ver,partido_4th_ver,votos_4th_ver
164,1,824,2178,ESCOLA DE ENFERMAGEM DE MANAUS - UFAM,ADRIANOPOLIS,-3.112116,-60.012732,326,241,85,228,98,54,AMAZONINO MENDES,PODE,41,CORONEL MENEZES,PATRIOTA,37,DAVID ALMEIDA,AVANTE,36,ZE RICARDO,PT,PSC,20,PSC,20,PSC,20,PSC,20
1404,40,533,1619,E. M. PROF. ELIANA LUCIA MONTEIRO DA SILVA,SANTO AGOSTINHO,-3.091425,-60.060595,392,299,93,286,106,73,AMAZONINO MENDES,PODE,58,DAVID ALMEIDA,AVANTE,43,CORONEL MENEZES,PATRIOTA,31,ZE RICARDO,PT,PMN,44,PMN,44,PMN,44,PMN,44
1875,59,610,2291,E. E. MARIA MADALENA SANTANA DE LIMA,ARMANDO MENDES,-3.089458,-59.942702,394,345,49,332,62,98,AMAZONINO MENDES,PODE,65,DAVID ALMEIDA,AVANTE,39,CORONEL MENEZES,PATRIOTA,38,RICARDO NICOLAU,PSD,AVANTE,31,AVANTE,31,AVANTE,31,AVANTE,31


### Adicionando o resultado da eleição para prefeito (2o turno)

In [116]:
df = df_resultados_prefeito_2t.merge(df_prefeitos_2t, on=['eleicao', 'cargo', 'numero'])
df.head()

,eleicao,zona,secao,local,cargo,numero,votos,partido,candidato
0,427,1,640,2429,Prefeito,19,156,PODE,AMAZONINO MENDES
1,427,1,641,2089,Prefeito,19,153,PODE,AMAZONINO MENDES
2,427,1,642,2321,Prefeito,19,138,PODE,AMAZONINO MENDES
3,427,1,643,2054,Prefeito,19,192,PODE,AMAZONINO MENDES
4,427,1,644,2402,Prefeito,19,137,PODE,AMAZONINO MENDES


In [117]:
# define o primeiro prefeito mais votado em cada seção (2o turno)
df_pref_2t_1st = get_nth_prefeito_mais_votado(df, 0)
df_pref_2t_1st.columns = ['zona', 'secao', 'votos_2t_1st', 'candidato_2t_1st', 'partido_2t_1st']
df_pref_2t_1st.sample(5)

,zona,secao,votos_2t_1st,candidato_2t_1st,partido_2t_1st
643,31,734,143,DAVID ALMEIDA,AVANTE
3240,70,672,165,DAVID ALMEIDA,AVANTE
2332,63,494,182,DAVID ALMEIDA,AVANTE
1277,40,406,147,AMAZONINO MENDES,PODE
1329,40,458,146,DAVID ALMEIDA,AVANTE


In [118]:
df_secoes_final = df_secoes_final.merge(df_pref_2t_1st, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th,partido_1st_ver,votos_1st_ver,partido_2nd_ver,votos_2nd_ver,partido_3rd_ver,votos_3rd_ver,partido_4th_ver,votos_4th_ver,votos_2t_1st,candidato_2t_1st,partido_2t_1st
2357,63,521,1724,E. M. DOM ADALBERTO MARZI,GILBERTO MESTRINHO,-3.062586,-59.925544,418,345,73,333,85,74,DAVID ALMEIDA,AVANTE,55,AMAZONINO MENDES,PODE,45,ZE RICARDO,PT,39,RICARDO NICOLAU,PSD,AVANTE,59,AVANTE,59,AVANTE,59,AVANTE,59,167,DAVID ALMEIDA,AVANTE
2031,62,329,1589,CMEI PROF. RAIMUNDO FIGUEIREDO,REDENCAO,-3.055647,-60.040741,394,328,66,317,77,68,DAVID ALMEIDA,AVANTE,60,AMAZONINO MENDES,PODE,47,CORONEL MENEZES,PATRIOTA,35,ZE RICARDO,PT,PATRIOTA,32,PATRIOTA,32,PATRIOTA,32,PATRIOTA,32,144,DAVID ALMEIDA,AVANTE
2586,63,753,1708,E. M. MOISES DE FRANCA VIANA,TANCREDO NEVES,-3.053639,-59.946413,423,311,112,297,126,57,AMAZONINO MENDES,PODE,53,DAVID ALMEIDA,AVANTE,49,ZE RICARDO,PT,38,CORONEL MENEZES,PATRIOTA,PATRIOTA,26,PATRIOTA,26,PATRIOTA,26,PATRIOTA,26,143,DAVID ALMEIDA,AVANTE


In [119]:
# define o segundo prefeito mais votado em cada seção (2o turno)
df_pref_2t_2nd = get_nth_prefeito_mais_votado(df, 0)
df_pref_2t_2nd.columns = ['zona', 'secao', 'votos_2t_2nd', 'candidato_2t_2nd', 'partido_2t_2nd']
df_pref_2t_2nd.sample(5)

,zona,secao,votos_2t_2nd,candidato_2t_2nd,partido_2t_2nd
2212,62,511,144,AMAZONINO MENDES,PODE
321,2,623,139,DAVID ALMEIDA,AVANTE
2311,62,624,120,DAVID ALMEIDA,AVANTE
1820,59,555,177,DAVID ALMEIDA,AVANTE
1579,58,842,162,AMAZONINO MENDES,PODE


In [120]:
df_secoes_final = df_secoes_final.merge(df_pref_2t_2nd, on=['zona', 'secao'])
df_secoes_final.sample(3)

,zona,secao,local,nome,bairro,lat,long,aptos,votos,abstencoes,votos_2t,abstencoes_2t,votos_1st,candidato_1st,partido_1st,votos_2nd,candidato_2nd,partido_2nd,votos_3rd,candidato_3rd,partido_3rd,votos_4th,candidato_4th,partido_4th,partido_1st_ver,votos_1st_ver,partido_2nd_ver,votos_2nd_ver,partido_3rd_ver,votos_3rd_ver,partido_4th_ver,votos_4th_ver,votos_2t_1st,candidato_2t_1st,partido_2t_1st,votos_2t_2nd,candidato_2t_2nd,partido_2t_2nd
1084,37,624,2003,E. E. LUCINDA FELIX DE AZEVEDO,JAPIIM,-3.110643,-59.986703,393,336,57,324,69,75,AMAZONINO MENDES,PODE,74,DAVID ALMEIDA,AVANTE,37,ZE RICARDO,PT,36,RICARDO NICOLAU,PSD,CIDADANIA,50,CIDADANIA,50,CIDADANIA,50,CIDADANIA,50,145,AMAZONINO MENDES,PODE,145,AMAZONINO MENDES,PODE
2691,65,423,1600,E. E. ROBERTO DOS SANTOS VIEIRA,NOVA CIDADE,-3.004946,-59.979299,420,355,65,331,89,77,DAVID ALMEIDA,AVANTE,74,AMAZONINO MENDES,PODE,43,ZE RICARDO,PT,43,RICARDO NICOLAU,PSD,PT,22,PT,22,PT,22,PT,22,143,AMAZONINO MENDES,PODE,143,AMAZONINO MENDES,PODE
2560,63,726,1732,E. M. ARMANDO MENDES,SAO JOSE OPERARIO,-3.067364,-59.944393,418,340,78,315,103,89,DAVID ALMEIDA,AVANTE,71,AMAZONINO MENDES,PODE,39,CORONEL MENEZES,PATRIOTA,38,RICARDO NICOLAU,PSD,PSC,38,PSC,38,PSC,38,PSC,38,160,DAVID ALMEIDA,AVANTE,160,DAVID ALMEIDA,AVANTE


### Gravando o arquivo csv final

In [121]:
filename = '/content/datasets/final_secoes.csv'
df_secoes_final.to_csv(filename)
print(f'Dataset final de seções gravado em \'{filename}\'')

Dataset final de seções gravado em '/content/datasets/final_secoes.csv'


In [122]:
# Limpeza do diretório temporário de trabalho
!rm -rf /content/work/*

## Perfil do Eleitorado
perfil_eleitor_secao_2020_AM.zip

In [ ]:
# perfil do eleitorado
!unzip -u perfil_eleitor_secao_2020_AM.zip -d /content/work

In [ ]:
df_perfil = pd.read_csv('/content/work/perfil_eleitor_secao_2020_AM.csv', encoding="ISO-8859-1", delimiter=';')
df_perfil.info()

In [ ]:
df_perfil = df_perfil[df_perfil.NM_MUNICIPIO.eq('MANAUS')]
df_perfil.head(5)

# Carga

In [123]:
%cd /content
!tar -czvf eleicoes2020.tar.gz datasets/*

/content
datasets/eleicoes.csv
datasets/escolas.csv
datasets/final_secoes.csv
datasets/leiame-boletim_urna.pdf
datasets/prefeitos_2t.csv
datasets/prefeitos.csv
datasets/resultados_prefeito_2t.csv
datasets/resultados_prefeito.csv
datasets/resultados_vereador.csv
datasets/secoes.csv
datasets/vereadores.csv


In [124]:
# sobrescreve o arquivo compartilhado: 1-1v_7xBem5VdYwFFaaNl2EKPLYaDadPa
!cp eleicoes2020.tar.gz /content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final

In [125]:
# remove o diretório
!rm -rf datasets

In [ ]:
!tar -zxvf eleicoes2020.tar.gz